In [0]:
#import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, DoubleType, TimestampType
from pyspark.sql.functions import *

# Configura as credenciais AWS a partir de variáveis de ambiente
spark = SparkSession.builder \
    .config("fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID")) \
    .config("fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY")) \
    .config("fs.s3a.endpoint", "s3.sa-east-1.amazonaws.com") \
    .config("fs.s3a.region", "sa-east-1") \
    .getOrCreate()

In [0]:
%sql

SHOW TABLES

database,tableName,isTemporary
default,taxi_consumo,false


In [0]:
df = spark.sql("SELECT * FROM taxi_consumo")

In [0]:
tipo_taxi = ["yellow", "green"]
print("\n=== Contagem total de registros ===")
print(f"Total de viagens: {df.count():,}")
print(f"Total de viagens frota Yellow: {df.filter(df.taxi_type == tipo_taxi[0]).count():,}")
print(f"Total de viagens forta Green: {df.filter(df.taxi_type == tipo_taxi[1]).count():,}")

print("\n=== Estatísticas Descritivas ===")
df.describe().show()

print("\n=== Estatísticas por VendorID ===")
df.groupBy("VendorID").agg(
    count("*").alias("num_viagens"),
    mean("total_amount").alias("valor_medio"),
    stddev("total_amount").alias("desvio_padrao_valor"),
    mean("passenger_count").alias("media_passageiros"),
    median("passenger_count").alias("mediana_passageiros")
).show()


=== Contagem total de registros ===
Total de viagens: 16,526,016
Total de viagens frota Yellow: 16,186,386
Total de viagens forta Green: 339,630

=== Estatísticas Descritivas ===
+-------+------------------+------------------+------------------+---------+------------------+
|summary|          VendorID|   passenger_count|      total_amount|taxi_type|        mes_pickup|
+-------+------------------+------------------+------------------+---------+------------------+
|  count|          16526016|          16526016|          16526016| 16526016|          16526016|
|   mean|1.7336136549789132|1.3504597841367212|27.730418327008117|     NULL|3.0769190832200572|
| stddev|0.4474873750706474|0.8818503906156275|22.833918395911294|     NULL|1.4154663190511665|
|    min|                 1|                 0|           -982.95|    green|                 1|
|    max|                 6|                 9|            6304.9|   yellow|                12|
+-------+------------------+------------------+-----

In [0]:
media_valor_yellow = df.filter(col("taxi_type") == "yellow") \
                      .groupBy("mes_pickup") \
                      .agg(round(mean("total_amount"),2).alias("media_valor")) \
                      .orderBy("mes_pickup")
media_valor_yellow.show()

+----------+-----------+
|mes_pickup|media_valor|
+----------+-----------+
|         1|      27.02|
|         2|       26.9|
|         3|       27.8|
|         4|      28.27|
|         5|      28.96|
|         6|      37.23|
|         7|      20.72|
|         8|      25.68|
|         9|      12.96|
|        10|      69.14|
|        11|      52.38|
|        12|      32.93|
+----------+-----------+



In [0]:
media_passageiros =  df.filter(month(col("pickup_datetime")) == 5) \
                       .groupBy(hour(col("pickup_datetime")).alias("hora_pickup")) \
                       .agg(round(avg("passenger_count"),2).alias("media_passageiros")) \
                       .orderBy("hora_pickup")

media_passageiros.show()

+-----------+-----------------+
|hora_pickup|media_passageiros|
+-----------+-----------------+
|          0|              1.4|
|          1|              1.4|
|          2|             1.42|
|          3|             1.41|
|          4|             1.35|
|          5|             1.24|
|          6|             1.22|
|          7|             1.24|
|          8|             1.25|
|          9|             1.27|
|         10|             1.31|
|         11|             1.32|
|         12|             1.34|
|         13|             1.35|
|         14|             1.35|
|         15|             1.36|
|         16|             1.36|
|         17|             1.35|
|         18|             1.35|
|         19|             1.36|
+-----------+-----------------+
only showing top 20 rows

